1. Read the data into a data frame. Make sure that the observation_date column is a datetime, which should then be set to the index. Which currencies have been tracked at the start of the Fed's data set? Which was the most recent currency to be added?

In [6]:
import pandas as pd

filename = '/Users/reuven/BambooWeekly/notebooks/data/bw-125.csv'

df = pd.read_csv(filename,
                parse_dates=['observation_date'],
                index_col='observation_date')

df

,DTWEXBGS,DEXUSEU,DEXJPUS,DEXCHUS,DEXCAUS,DEXUSUK,DEXKOUS,DEXMXUS,DEXVZUS,DTWEXAFEGS,DEXINUS,DEXBZUS
observation_date,,,,,,,,,,,,
1971-01-04,NaN,NaN,357.73,NaN,1.0109,2.3938,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-05,NaN,NaN,357.81,NaN,1.0102,2.3949,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-06,NaN,NaN,357.86,NaN,1.0106,2.3967,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-07,NaN,NaN,357.87,NaN,1.0148,2.3963,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-08,NaN,NaN,357.82,NaN,1.0154,2.3972,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-23,121.5412,1.1538,146.47,7.1790,1.3773,1.3491,1382.61,19.1600,104.2820,111.8977,86.73,5.5163
2025-06-24,120.7549,1.1608,144.90,7.1713,1.3711,1.3616,1359.00,19.0210,104.4127,111.1217,85.92,5.4985
2025-06-25,120.7333,1.1620,145.59,7.1764,1.3750,1.3626,1362.17,18.9159,104.4127,111.2173,86.04,5.5589


In [7]:
df.dtypes

DTWEXBGS      float64
DEXUSEU       float64
DEXJPUS       float64
DEXCHUS       float64
DEXCAUS       float64
DEXUSUK       float64
DEXKOUS       float64
DEXMXUS       float64
DEXVZUS       float64
DTWEXAFEGS    float64
DEXINUS       float64
DEXBZUS       float64
dtype: object

In [11]:
# Which currencies have been tracked at the start of the Fed's data set? 

(
    df
    .filter(regex='^DEX')  # show only columns in which the column name starts with DEX
    .iloc[0]
    .dropna()
    .index
)

Index(['DEXJPUS', 'DEXCAUS', 'DEXUSUK'], dtype='object')

In [ ]:
# Which was the most recent currency to be added?

In [14]:
(
    df
    .filter(regex='^DEX')
    .apply(lambda c_: c_.first_valid_index(), axis='rows')
    .sort_values()
)

DEXJPUS   1971-01-04
DEXCAUS   1971-01-04
DEXUSUK   1971-01-04
DEXINUS   1973-01-02
DEXCHUS   1981-01-02
DEXKOUS   1981-04-13
DEXMXUS   1993-11-08
DEXVZUS   1995-01-02
DEXBZUS   1995-01-02
DEXUSEU   1999-01-04
dtype: datetime64[ns]

2. The New York Times article said, "The dollar is off to its worst start to a year in more than half a century." Calculate the percentage drop in the "broad" index (DTWEXBGS) for the first six months of each year starting in 2020.

In [24]:
(
    df
    .loc['2020':, 'DTWEXBGS']  # two-argument version of .loc is [row_selector, column_selector]
    .dropna()
    .reset_index()
    .assign(year = lambda df_: df_['observation_date'].dt.year,
            month = lambda df_: df_['observation_date'].dt.month)
    .loc[lambda df_: df_['month'].isin(range(7))]
    .groupby('year')['DTWEXBGS'].agg(['first', 'last'])
    .pct_change(axis='columns')
    ['last']
    .sort_values()
)

year
2025   -0.073943
2023   -0.019434
2020   -0.001666
2021    0.012112
2024    0.040970
2022    0.048062
Name: last, dtype: float64